# 1.Data Import and Combination

In [113]:
import pandas as pd
import gensim as gs
import nltk
import re
from nltk.corpus import stopwords
import sklearn as sk
import numpy as np
import pickle
from tqdm import tqdm

## 1.1.import dataset

In [114]:
train_path="DataSet\\train.txt"
test_path="DataSet\\test.txt"

train_df = pd.read_csv(train_path,sep="\t",index_col=0)
test_df=pd.read_csv(test_path,sep="\t",index_col=0)


In [115]:
train_df

,turn1,turn2,turn3,label
id,,,,
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others


In [116]:
test_df

,turn1,turn2,turn3,label
id,,,,
0,Hmm,What does your bio mean?,I don’t have any bio,others
1,What you like,very little things,Ok,others
2,Yes,How so?,I want to fuck babu,others
3,what did you guess,what what,fuck,others
4,We ?,of course we will!,What gender movies you like??,others
...,...,...,...,...
5504,Not youuu,I also didn't not not.,How to calll,others
5505,Welcome,"Why, thank you.","I don't know, you tell",others
5506,Yes,IF ONLY I COULD AFFORD THIS,How are you,others


## 1.2.unify the variables for change

In [117]:
# now when I copy the code for another dataset, I can easily change the special name of the dataset for adjusting the process and storage
process_df = test_df
version_change = "glove"
path_change = "test"

In [118]:
emo_path="DataSet\\emo_collection_{b}.csv".format(b=version_change)
emo_df=pd.read_csv(emo_path,sep="|")

In [119]:
emo_df

,emoji/emoticon,key words
0,😀,face grin grinning face
1,😃,face grinning face big eyes mouth open smile
2,😄,eye face grinning face smiling eyes mouth ope...
3,😁,beaming face smiling eyes eye face grin smile
4,😆,face grinning squinting face laugh mouth sati...
...,...,...
2279,( ´Д｀)ﾉ(´･ω･`) ﾅﾃﾞﾅﾃﾞ,patting made made
2280,(*ﾟﾉOﾟ)<ｵｵｵｵｫｫｫｫｫｫｫｰｰｰｰｰｲ!,calling ooooh
2281,( ﾟ∀ﾟ)ｱﾊﾊ八八ﾉヽﾉヽﾉヽﾉ ＼ / ＼/ ＼,evil laugh literally ahahahaha
2282,（・∀・ ）ヾ(- -；)コラコラ,blaming


## 1.3.get label and turn into category

In [120]:
# set the label into categorical number:
# others:0; happy:1,angry:3;sad:2;

class Label_format:
    def __init__(self,df):
        self.df=df
        
    def get_y(self):
        y = self.df.iloc[:,-1].values
        return y
    
    def label_category(self,y):
        change_list = list()
        emotion = {"others":0,"happy":1,"sad":2,"angry":3}
        for each in y:
            change_list.append(emotion[each])
        change_array=np.array(change_list)
        return change_array
    
    def save_file(self,data,name):
        pickle_file = open('DataSet\\'+name,mode='wb')
        pickle.dump(data,pickle_file)
        pickle_file.close()

In [121]:
#set y label, and we make X later on.

lf = Label_format(process_df)
y = lf.get_y()
y = lf.label_category(y)
save_path = "{a}_y.pkl".format(a=path_change)
lf.save_file(y,save_path)

In [122]:
y

array([0, 0, 0, ..., 0, 0, 0])

## 1.4. class for combination

In [123]:
# because we use pre-trained embeddings, we can't directly use stemming, remove stopwords or remove punctuations.
# we need to turn the data as much as possible close to the pre-trained embedding.
def basic_clean_text(input_text):
    'clean text function'
    #clean emoji, punctions
    input_text = re.sub(r'[^a-zA-Z]',repl=" ",string=input_text)
    #lower the words
    input_text =input_text.lower()
    #stemming the words
    list_text=input_text.split()
    ps=nltk.stem.porter.PorterStemmer()
    list_text_out=list()
    for word in list_text:
        if word in set(stopwords.words('english')):
            continue
        list_text_out.append(ps.stem(word))
    input_text=" ".join(list_text_out) 
    return input_text

def combine_method(df):
    turn1_corpus= list(df["turn1"])
    turn2_corpus= list(df["turn2"])
    turn3_corpus= list(df["turn3"])
    
    # use text_content as dict for storing the data of 
    text_content={"turn1":turn1_corpus,"turn2":turn2_corpus,"turn3":turn3_corpus}
    
    # because we need all the sentences including sentences in turn1, turn2, turn3 
    # to predict the emtion situation, therefore we need to combine them together
    length = len(text_content["turn1"])
    text_all_content = list()
    for i in tqdm(range(length)):
        text_row_all = text_content["turn1"][i] +" " + text_content["turn2"][i] +" "+ text_content["turn3"][i]
        text_all_content.append(text_row_all)
    text_all_content = {"all":text_all_content}
    
    return text_content,text_all_content

In [124]:
text_3,text_all = combine_method(process_df)

100%|█████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 1104513.42it/s]


In [125]:
text_3

{'turn1': ['Hmm',
  'What you like',
  'Yes',
  'what did you guess',
  'We ?',
  'Where are you now?',
  'That was a joke btw...',
  'Who d hell s he',
  'yes, good advice',
  'Nice to meet u',
  'Yupp',
  'Software',
  'Very nice ',
  'First  you  hurt me',
  'Love you 🙊',
  'In India Fogg is going on',
  'is my grammar perfect or should I need to learn more',
  'by',
  'I don’t know what to write',
  'so you make jokes. i already got that.',
  'Yeah mee too',
  'Thanks',
  'Offer?',
  'Uhhhmm whyy??',
  'sure will u cal me ni8',
  'About what I just told',
  'Yeaaa black panther',
  'now dont give me a chance to be an danger evil',
  'Watch spitsvilla at 7 p.m today',
  'Uuu',
  "But you do 'I can't want any of this'",
  'Ok',
  'Purple',
  'You wont get it',
  'Thn tell me',
  'Why alright?',
  'Do you like video games?',
  'Buy',
  'Just asking',
  'not possible',
  'Your photo please',
  'Oh sorry',
  'What is press box',
  'So what?',
  'I already heard of ir',
  'And you are my

In [126]:
text_all

{'all': ['Hmm What does your bio mean? I don’t have any bio',
  'What you like very little things Ok ',
  'Yes How so? I want to fuck babu',
  'what did you guess what what fuck',
  'We ? of course we will! What gender movies you like??',
  'Where are you now? At home just about to have breakfast... what are you eating?',
  'That was a joke btw... it was Yes 😁',
  'Who d hell s he johnny depp...duh? Who she',
  'yes, good advice best advice ... i great thx',
  'Nice to meet u Hi, nice to meet you too! 😸😂 😁',
  "Yupp why? Don't know I'm tired",
  'Software Software what? I plan on going for development next year I am into android development stuff',
  'Very nice  Thanks!! :) R u know tamil ',
  'First  you  hurt me okay So I talked  rude',
  "Love you 🙊 you don't recognize me? 😏 I love you 🙊",
  'In India Fogg is going on MumbaiIndians getting routed!!! Ohh really',
  'is my grammar perfect or should I need to learn more Yes, it is possible. thank you',
  'by In Suits. have good day',
 

## 1.5. Main Function

In [127]:
def fit_transform(input_dict,method):
    output_dict=dict()
    for key in input_dict.keys():
        processed_texts = list()
        for text in tqdm(input_dict[key]):
            processed_single_text = eval(method)(text)
            processed_texts.append(processed_single_text)
        
        output_dict[key]=processed_texts
    
    return output_dict

# 2. Preprocessing for cleaning the data
## 2.1. Extract emoji, emoticons based on emo_df, and build 2 versions of data
- one version for saving the extracted emoji, emoticons of each sentence
- one version for text without the emoji, emoticons based on emo_df

In [128]:
emo_values = list(emo_df["emoji/emoticon"])
meaning_values = list(emo_df["key words"])

def count_emo(sentence,emo):
    'function for counting how many same emoji or emoticon in one sentence'
    sentence = sentence.replace(emo," count_no ")
    words_list = sentence.split()
    count_no=0
    for word in words_list:
        if "count_no" in word:
            count_no +=1
    
    return count_no
            

def extract_emo(text_dict,emo_values,meaning_values):
    calculate_emo=dict()
    del_emo_dict=dict()
    for key in text_dict.keys():
        length = len(text_dict[key])
        del_emo_list = text_dict[key].copy()
        
        # set list_all_emo as the storage for storing all emoji and emoticons for all sentences
        list_all_emo = list()

        len_emo = len(emo_values)
        for i in tqdm(range(length)):
            
            # set list_emo as the storage for storing the emo for each sentence
            list_emo = list()
            for e in range(len_emo):
                if emo_values[e] in text_dict[key][i]:
                    count_no = count_emo(text_dict[key][i],emo_values[e])
                    list_emo.append([count_no,meaning_values[e]])
                    del_emo_list[i]=del_emo_list[i].replace(emo_values[e], " ")
            
            list_all_emo.append(list_emo)

        calculate_emo[key]=list_all_emo
        del_emo_dict[key]=del_emo_list
    
    return del_emo_dict,calculate_emo


In [129]:
# save the storage of emoji and emoticons
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [130]:
text_3_0, calculate_emo_3 = extract_emo(text_3,emo_values,meaning_values)
text_all_0, calculate_emo_all = extract_emo(text_all,emo_values,meaning_values)

100%|████████████████████████████████████████████████████████████████████████████| 5509/5509 [00:01<00:00, 4685.10it/s]


In [131]:
calculate_emo_3

{'turn1': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' evil face forbidden monkey speak speak no evil monkey ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' face sleep sleepy face ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' night night stars star ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' disappointed face ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
 

In [132]:
save_path = "extraction_emo_3_{a}_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path,calculate_emo_3)

In [133]:
text_3_0

{'turn1': ['Hmm',
  'What you like',
  'Yes',
  'what did you guess',
  'We ?',
  'Where are you now?',
  'That was a joke btw...',
  'Who d hell s he',
  'yes, good advice',
  'Nice to meet u',
  'Yupp',
  'Software',
  'Very nice ',
  'First  you  hurt me',
  'Love you  ',
  'In India Fogg is going on',
  'is my grammar perfect or should I need to learn more',
  'by',
  'I don’t know what to write',
  'so you make jokes. i already got that.',
  'Yeah mee too',
  'Thanks',
  'Offer?',
  'Uhhhmm whyy??',
  'sure will u cal me ni8',
  'About what I just told',
  'Yeaaa black panther',
  'now dont give me a chance to be an danger evil',
  'Watch spitsvilla at 7 p.m today',
  'Uuu',
  "But you do 'I can't want any of this'",
  'Ok',
  'Purple',
  'You wont get it',
  'Thn tell me',
  'Why alright?',
  'Do you like video games?',
  'Buy',
  'Just asking',
  'not possible',
  'Your photo please',
  'Oh sorry',
  'What is press box',
  'So what?',
  'I already heard of ir',
  'And you are my

# 2.2. Lower the words

In [134]:
# lower the words
def lower_text(input_text):
    #lower the words
    input_text =input_text.lower()
    return input_text

In [135]:
text_3_1 = fit_transform(text_3_0,"lower_text")
text_all_1 = fit_transform(text_all_0,"lower_text")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 183890.72it/s]


## 2.3. Seperate the adhered words with space and remove redundant emoji, emoticons
 - e.g. "you?" ? and you are adhered together, which needs to be split
 - e.g. "yess i am crazyyy😂😂" I also need to split emoji from the word "crazyyy" as "i am crazyyy 😂 😂 "
 - and some emoji, emoticons can't be found in the emo_df, therefore, I delete them since it's impossible to explain them in word2vec

In [136]:
# functions for extracting emoji, emoticons and punctuations
def extract_emoji(input_text):
    emoji_list = re.findall(r'(?:[^a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
    return emoji_list
    
def extract_emoticon(input_text):
    emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
    return emoticon_list

def extract_punctuation(input_text):
    'here I use for loop to split the double punctuations into single ones'
    punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
    result_list=list()
    for each in punctuation_list:
        list_split=[each]
        if each !="...":
            list_split=list(each)
        for punc in list_split:
            result_list.append(punc)
    return result_list

# functions for assigning emoji, emoticons and punctuations space from each other
def assign_space(input_text):
    #emoji
    input_text = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',' ',input_text)
    #emoticon
    input_text = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',' ',input_text)
    #punctuation
    input_text = re.sub(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',' \g<0> ' ,input_text)
    #replace the redundant space
    input_text = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
    input_text = re.sub(r'  +',r' ',input_text)
    return input_text

In [137]:
text_3_2 = fit_transform(text_3_1,"assign_space")
text_all_2 = fit_transform(text_all_1,"assign_space")

100%|███████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 27210.47it/s]


# 3. Preprocessing: Correct the words in sentence
## 3.1. Correct common words

In [138]:
# turn don't, doesn't, can't, couldn't, shouldn't, didn't, isn't, wasn't, weren't, aren't, won't, shan't, 's, 're
# as do not, do not, can not, could not, should not, did not, is not, was not, were not, are not, will not, shall not, 's, are

# turn r , u, bt, n, 
# as are, you ,but, and
cor_dict= {
    "don ' t":      "do not",
    "doesn ' t":    "do not",
    "can ' t":      "can not",
    "couldn ' t":   "could not",
    "shouldn ' t":  "should not",
    "didn ' t":     "did not",
    "isn ' t":      "is not",
    "wasn ' t":     "was not",
    "weren ' t":    "were not",
    "aren ' t":     "are not",
    "won ' t":      "will not",
    "shan ' t":     "shall not",
    
    #"\' re ":       "are ",
    #" re ":         "are",
    "\' m ":        "am ",
    "\' ll ":       "will ",
    #" \' s ":       " \'s "

    }

In [139]:
def correct_words_common(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = input_text
    for each_key in cor_dict.keys():
        if each_key in input_text:
            corrected_sentence = corrected_sentence.replace(each_key,cor_dict[each_key])
    return corrected_sentence


In [140]:
text_3_3 = fit_transform(text_3_2,"correct_words_common")
text_all_3 = fit_transform(text_all_2,"correct_words_common")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 137967.73it/s]


## 3.2. Correct missing characters
- e.g. "hmmm i can talk no w", here "now" is missing "w".

In [141]:
def correct_words_combin(input_text):
    corrected_sentence = input_text
    corrected_sentence = re.sub(r' ([a-z]+) ([bcdefghjklnopqrstvwxz]) ',r' \g<1>\g<2> ',corrected_sentence)
    return corrected_sentence

In [142]:
testset = {"turn1":["I ' m go d , and I wasn ' t player wai t ", "hhh you ' re kidding m e "]}
print(fit_transform(testset,"correct_words_combin"))

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2007.32it/s]

{'turn1': ["I ' m god , and I wasn ' t player wait ", "hhh you ' re kidding me "]}


In [143]:
text_3_4 = fit_transform(text_3_3,"correct_words_combin")
text_all_4 = fit_transform(text_all_3,"correct_words_combin")

100%|███████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 89092.55it/s]


In [144]:
text_all_4["all"][:10]

[' hmm what does your bio mean ? i dont have any bio ',
 ' what you like very little things ok ',
 ' yes hows o ? i want to fuck babu ',
 ' what did you guess what what fuck ',
 ' we ? of course we will ! what gender movies you like ',
 ' where are you now ? at home just about to have breakfast what are you eating ?',
 ' that was a joke btw it was yes ',
 ' whod hell s he johnny depp duh ? who she ',
 ' yes , good advice best advice i great thx ',
 ' nice to meet u hi , nice to meet you too ! ']

## 3.3. Remove punctuations

In [145]:
def remove_punc(input_text):
    new_sentence = input_text
    new_sentence = re.sub(r'[^ a-zA-Z]',"",new_sentence).strip()
    return new_sentence

In [146]:
text_3_5 = fit_transform(text_3_4,"remove_punc")
text_all_5 = fit_transform(text_all_4,"remove_punc")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 134728.17it/s]


In [147]:
text_all_5["all"][:10]

['hmm what does your bio mean  i dont have any bio',
 'what you like very little things ok',
 'yes hows o  i want to fuck babu',
 'what did you guess what what fuck',
 'we  of course we will  what gender movies you like',
 'where are you now  at home just about to have breakfast what are you eating',
 'that was a joke btw it was yes',
 'whod hell s he johnny depp duh  who she',
 'yes  good advice best advice i great thx',
 'nice to meet u hi  nice to meet you too']

## 3.4.  Remove stopwords made handcrafted

In [148]:
print(stopwords.words('english')) # which is not suitable here, coz e.g. in emotion analysis, not happy is opposite from happy.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [149]:
stopwords_byhand = []
# since combination of words can make the sentiment totally different
# e.g. "wasn't happy" is more likely to be sad, not happy.
# e.g. "you're sad" doesn't mean "I'm sad"
# therefore in order not to mix the order of the sentence, I try to keep those stopwords in sentence
# besides, when doing CNN or LSTM, it's important to detect all the structure of the sentence

In [150]:
def remove_stopwords(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        if word in stopwords_byhand:
            continue
        else:
            new_words.append(word)
    sentence = " ".join(new_words)
  
    return sentence

In [151]:
text_3_6 = fit_transform(text_3_5,"remove_stopwords")
text_all_6 = fit_transform(text_all_5,"remove_stopwords")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 128459.64it/s]


# 4. Preprocessing: Correction of Incorrect Spell and Shorthand Words
## 4.1. Correcting spell mistake
 - e.g. "nt" is actually "not", "noo" is actually "no"
 - in order to do so, here i use library called pyspellchecker

In [152]:
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spell(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        word = spell.correction(word)
        new_words.append(word)
    sentence = " ".join(new_words)
    return sentence


In [153]:
text_3_7 = fit_transform(text_3_6,"correct_spell")
text_all_7 = fit_transform(text_all_6,"correct_spell")

100%|██████████████████████████████████████████████████████████████████████████████| 5509/5509 [04:42<00:00, 19.50it/s]


In [154]:
text_all_7 ["all"][:10]

['hmm what does your bio mean i dont have any bio',
 'what you like very little things ok',
 'yes hows o i want to fuck baby',
 'what did you guess what what fuck',
 'we of course we will what gender movies you like',
 'where are you now at home just about to have breakfast what are you eating',
 'that was a joke btw it was yes',
 'who hell s he johnny deep due who she',
 'yes good advice best advice i great the',
 'nice to meet u hi nice to meet you too']

In [155]:
# after spell mistakce correction, it's needed to run over again to remove the stopwords
text_3_8 = fit_transform(text_3_7,"remove_stopwords")
text_all_8 = fit_transform(text_all_7,"remove_stopwords")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 112729.64it/s]


## 4.2. shorthand words translation
- here we only deal with the most frequent words happened in text which can't be changed into vectors based on pretrained word2vec index

In [156]:
cor_dict2= {
    "whatsaap":"whatsapp",
    "whatsappn":"whatsapp",
    "ohhhooo":"oh oh",
    "tel-me":"tell me",
    "hahahahhahh":"hahaha",
    "okkkkkkkk":"ok",
    "heeaaadd":"head",
    "happyhalloween":"happy halloween",
    "hahhhahh":"hahaha",
    "amazingggggggg":"amazing",
    "whtsapp":"whatsapp",
    "thankyouuuu":"thank you",
    }

In [157]:
def correct_words_common2(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = " "+input_text+" "
    for each_key in cor_dict2.keys():
        if each_key in input_text:
            rule = " "+each_key+" "
            corrected_sentence = re.sub(rule," "+cor_dict2[each_key]+" ",corrected_sentence).strip()
            #corrected_sentence.replace(each_key,cor_dict2[each_key])
    return corrected_sentence

In [158]:
text_3_9 = fit_transform(text_3_8,"correct_words_common2")
text_all_9 = fit_transform(text_all_8,"correct_words_common2")

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 141638.14it/s]


# 5. check data format 
- for further preprocessing based on pre-trained embedding resource

## 5.1. check distinct vocabulary

In [159]:
def distinct_words(dialogs):
    vocab = dict()
    for each_dialog in tqdm(dialogs):
        words_list = each_dialog.split(sep=" ")
        for word in words_list:
            if word == '':
                continue
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    
    return vocab

In [160]:
vocab = distinct_words(text_all["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version original",b=len(vocab)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 250628.25it/s]

the length of distinct vocabulary of the version original is 10814


In [161]:
vocab0 = distinct_words(text_all_0["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 0",b=len(vocab0)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 212451.35it/s]

the length of distinct vocabulary of the version 0 is 10331


In [162]:
vocab1 = distinct_words(text_all_1["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 1",b=len(vocab1)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 250769.69it/s]

the length of distinct vocabulary of the version 1 is 8763


In [163]:
vocab2 = distinct_words(text_all_2["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 2",b=len(vocab2)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 220773.93it/s]

the number of distinct vocabulary of the version 2 of data is 6155


In [164]:
vocab3 = distinct_words(text_all_3["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 3",b=len(vocab3)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 230155.09it/s]

the number of distinct vocabulary of the version 3 of data is 6151


In [165]:
vocab4 = distinct_words(text_all_4["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 4",b=len(vocab3)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 220944.93it/s]

the number of distinct vocabulary of the version 4 of data is 6151


In [166]:
vocab5 = distinct_words(text_all_5["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 5",b=len(vocab5)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 276194.37it/s]

the number of distinct vocabulary of the version 5 of data is 5877


In [167]:
vocab6 = distinct_words(text_all_6["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 6",b=len(vocab6)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 276181.16it/s]

the number of distinct vocabulary of the version 6 of data is 5877


In [168]:
vocab7 = distinct_words(text_all_7["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 7",b=len(vocab7)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 276194.37it/s]

the number of distinct vocabulary of the version 7 of data is 4899


In [169]:
vocab8 = distinct_words(text_all_8["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 8",b=len(vocab8)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 276187.76it/s]

the number of distinct vocabulary of the version 8 of data is 4899


In [170]:
vocab9 = distinct_words(text_all_9["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 9",b=len(vocab9)))

100%|██████████████████████████████████████████████████████████████████████████| 5509/5509 [00:00<00:00, 262534.18it/s]

the number of distinct vocabulary of the version 9 of data is 4896


## 5.2. check the percentage of words in data can be processed by word2vec pre-trained data

In [171]:
# import processed words list from glove
GloVe_path="DataSet\\glove.840B.300d_words.pkl"
def open_pkl(path):
    pickle_file = open(path,mode='rb')
    data = pickle.load(pickle_file)
    pickle_file.close()

    return data

def form_dict(words_list):
    words_dict = dict()
    content="just for data check"
    for word in words_list:
        words_dict[word]=content
    
    return words_dict

In [172]:
glo_words=open_pkl(GloVe_path)
glo_dict = form_dict(glo_words)# because process dict is much faster than processing list

In [173]:
glo_dict["many"]

'just for data check'

In [174]:
# build the function for check the coverage of the vocabulary and the text
import operator
def check_coverage(vocab, embedding_resource):
    cov_vocab =0
    num_vocab = len(vocab)
    cov_text = 0
    not_cov = dict()
    not_text = 0
    for word in tqdm(vocab):
        try:
            x = embedding_resource[word]
            cov_vocab += 1
            cov_text += vocab[word]
        except:
            not_cov[word]=vocab[word]
            not_text += vocab[word]
            pass
    percent_cov_vocab = cov_vocab/num_vocab
    percent_cov_text = cov_text/(cov_text+not_text)
    print("In Embedding Index we have {:.2%} coverage of distinct vocabulary".format(percent_cov_vocab))
    print("And we have {:.2%} coverage of all text".format(percent_cov_text))
    sorted_not_cov = sorted(not_cov.items(),key= operator.itemgetter(1),reverse = True)
    print("The number of words which are not covered in word2vec resource is: {0}".format(len(sorted_not_cov)))
    return sorted_not_cov

In [175]:
# original version of data
not_cov = check_coverage(vocab, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 10814/10814 [00:00<00:00, 1200084.76it/s]

In Embedding Index we have 62.83% coverage of distinct vocabulary
And we have 90.35% coverage of all text
The number of words which are not covered in word2vec resource is: 4020


In [176]:
not_cov[:20]

[('you?', 108),
 ('what?', 98),
 ('😂', 72),
 ("That's", 61),
 ("You're", 55),
 ('me?', 50),
 ("What's", 48),
 (':(', 45),
 ('it?', 42),
 ('why?', 41),
 ("what's", 32),
 ('that?', 29),
 ('Why?', 28),
 ('u?', 27),
 ('What?', 26),
 ('..', 26),
 ("won't", 25),
 ('you!', 24),
 ('😁', 22),
 ("i'm", 21)]

In [177]:
# remove emoji, emoticon
not_cov0 = check_coverage(vocab0, glo_dict)

100%|███████████████████████████████████████████████████████████████████████| 10331/10331 [00:00<00:00, 1150960.33it/s]

In Embedding Index we have 65.50% coverage of distinct vocabulary
And we have 91.59% coverage of all text
The number of words which are not covered in word2vec resource is: 3564


In [178]:
not_cov0[:20]
# here we can see that the trained word2vec resource can't cover some stopwords, punctuations and the combination with word and punctuation

[('you?', 109),
 ('what?', 98),
 ("That's", 61),
 ("You're", 55),
 ('me?', 51),
 ("What's", 48),
 ('it?', 42),
 ('why?', 41),
 ("what's", 32),
 ('that?', 29),
 ('Why?', 29),
 ('What?', 28),
 ('u?', 27),
 ('..', 27),
 ("won't", 25),
 ('you!', 25),
 ("i'm", 21),
 ("isn't", 21),
 ('so?', 20),
 ('now?', 20)]

In [179]:
# lower the words
not_cov1 = check_coverage(vocab1, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 8763/8763 [00:00<00:00, 1250244.44it/s]

In Embedding Index we have 62.19% coverage of distinct vocabulary
And we have 91.20% coverage of all text
The number of words which are not covered in word2vec resource is: 3313


In [180]:
not_cov1[:20]
# here we can see that word2vec resource can't cover the words that are not correct spelled.

[("i'm", 415),
 ('what?', 128),
 ('you?', 119),
 ("what's", 80),
 ("i'll", 78),
 ('why?', 72),
 ('me?', 53),
 ('it?', 43),
 ('u?', 30),
 ('that?', 29),
 ('no,', 28),
 ("let's", 28),
 ('you!', 28),
 ('..', 27),
 ('yes,', 26),
 ("won't", 26),
 ("i've", 26),
 ('so?', 22),
 ("isn't", 22),
 ("haven't", 22)]

In [181]:
# seperate adhered words such as "you!!!" with space into "you ! ! ! "
not_cov2 = check_coverage(vocab2, glo_dict)

100%|██████████████████████████████████████████████████████████████████████████| 6155/6155 [00:00<00:00, 881752.21it/s]

In Embedding Index we have 92.25% coverage of distinct vocabulary
And we have 99.38% coverage of all text
The number of words which are not covered in word2vec resource is: 477


In [182]:
not_cov2[:20] #now we can see the adhered words pair such as "you!!!" are gone

[('nevermin', 4),
 ('ntng', 4),
 ('erienc', 3),
 (',,', 3),
 ('2,', 3),
 ('useropenreflink', 3),
 ('sarcasti', 3),
 ('chathero', 2),
 ('sachhi', 2),
 ('lohiya', 2),
 ('bollywoo', 2),
 ('autowala', 2),
 ('byyyy', 2),
 ('ursel', 2),
 ('&,', 2),
 ('ridiculou', 2),
 ('patienc', 2),
 ('.,', 2),
 ('cauliflowe', 2),
 ('hallowee', 2)]

In [183]:
# correct common words e.g. "you're" as "you are"
not_cov3 = check_coverage(vocab3, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 6151/6151 [00:00<00:00, 1233524.45it/s]

In Embedding Index we have 92.25% coverage of distinct vocabulary
And we have 99.37% coverage of all text
The number of words which are not covered in word2vec resource is: 477


In [184]:
not_cov3[:20]

[('nevermin', 4),
 ('ntng', 4),
 ('erienc', 3),
 (',,', 3),
 ('2,', 3),
 ('useropenreflink', 3),
 ('sarcasti', 3),
 ('chathero', 2),
 ('sachhi', 2),
 ('lohiya', 2),
 ('bollywoo', 2),
 ('autowala', 2),
 ('byyyy', 2),
 ('ursel', 2),
 ('&,', 2),
 ('ridiculou', 2),
 ('patienc', 2),
 ('.,', 2),
 ('cauliflowe', 2),
 ('hallowee', 2)]

In [185]:
# add missing charactors e.g. "hmmm i can talk no w", here "now" is missing "w".
not_cov4 = check_coverage(vocab4, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 5933/5933 [00:00<00:00, 1189920.41it/s]

In Embedding Index we have 90.90% coverage of distinct vocabulary
And we have 99.25% coverage of all text
The number of words which are not covered in word2vec resource is: 540


In [186]:
not_cov4[:20]

[('howr', 7),
 ('goingo', 4),
 ('ntng', 4),
 ('offt', 4),
 ('erienc', 3),
 ('wheret', 3),
 ('whyr', 3),
 (',,', 3),
 ('2,', 3),
 ('snown', 3),
 ('useropenreflink', 3),
 ('chathero', 2),
 ('sachhi', 2),
 ('lohiya', 2),
 ('autowala', 2),
 ('shallv', 2),
 ('byyyy', 2),
 ('&,', 2),
 ('eachn', 2),
 ('.,', 2)]

In [187]:
# remove punctuation
not_cov5 = check_coverage(vocab5, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 5877/5877 [00:00<00:00, 1473396.57it/s]

In Embedding Index we have 90.95% coverage of distinct vocabulary
And we have 99.20% coverage of all text
The number of words which are not covered in word2vec resource is: 532


In [188]:
not_cov5[:20]

[('howr', 7),
 ('goingo', 4),
 ('ntng', 4),
 ('offt', 4),
 ('erienc', 3),
 ('wheret', 3),
 ('whyr', 3),
 ('snown', 3),
 ('useropenreflink', 3),
 ('chathero', 2),
 ('sachhi', 2),
 ('lohiya', 2),
 ('autowala', 2),
 ('shallv', 2),
 ('byyyy', 2),
 ('eachn', 2),
 ('jabh', 2),
 ('wachapp', 2),
 ('wlcm', 2),
 ('happyb', 2)]

In [189]:
# Remove stopwords made handcrafted
not_cov6 = check_coverage(vocab6, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 5877/5877 [00:00<00:00, 1178576.36it/s]

In Embedding Index we have 90.95% coverage of distinct vocabulary
And we have 99.20% coverage of all text
The number of words which are not covered in word2vec resource is: 532


In [190]:
not_cov6[:20]

[('howr', 7),
 ('goingo', 4),
 ('ntng', 4),
 ('offt', 4),
 ('erienc', 3),
 ('wheret', 3),
 ('whyr', 3),
 ('snown', 3),
 ('useropenreflink', 3),
 ('chathero', 2),
 ('sachhi', 2),
 ('lohiya', 2),
 ('autowala', 2),
 ('shallv', 2),
 ('byyyy', 2),
 ('eachn', 2),
 ('jabh', 2),
 ('wachapp', 2),
 ('wlcm', 2),
 ('happyb', 2)]

In [191]:
# correct misspell
not_cov7 = check_coverage(vocab7,glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 4899/4899 [00:00<00:00, 1216067.66it/s]

In Embedding Index we have 97.51% coverage of distinct vocabulary
And we have 99.83% coverage of all text
The number of words which are not covered in word2vec resource is: 122


In [192]:
not_cov7[:20]

[('useropenreflink', 3),
 ('wachapp', 2),
 ('waitinnnnnnnn', 2),
 ('mumbaiindians', 1),
 ('hh:mm', 1),
 ('spitsvilla', 1),
 ('okkkkkkkk', 1),
 ('aaagdghsuuqiosjhsubdhuskqondjis', 1),
 ('whatham', 1),
 ('marryat', 1),
 ('ershad', 1),
 ('playcreepypedia', 1),
 ('feelingyusohot', 1),
 ('ialsowithyu', 1),
 ('virtualasstftw', 1),
 ('fuuuuuuukkkhhhhh', 1),
 ('yuuuuuuuuuuuuuuuuuuuuuuuu', 1),
 ('sharradha', 1),
 ('mommmmmmmm', 1),
 ('ummmmaaaah', 1)]

In [193]:
# remove stopwords again after correction of missepll
not_cov8 = check_coverage(vocab8, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 4899/4899 [00:00<00:00, 1227986.33it/s]

In Embedding Index we have 97.51% coverage of distinct vocabulary
And we have 99.83% coverage of all text
The number of words which are not covered in word2vec resource is: 122


In [194]:
not_cov8[:50]

[('useropenreflink', 3),
 ('wachapp', 2),
 ('waitinnnnnnnn', 2),
 ('mumbaiindians', 1),
 ('hh:mm', 1),
 ('spitsvilla', 1),
 ('okkkkkkkk', 1),
 ('aaagdghsuuqiosjhsubdhuskqondjis', 1),
 ('whatham', 1),
 ('marryat', 1),
 ('ershad', 1),
 ('playcreepypedia', 1),
 ('feelingyusohot', 1),
 ('ialsowithyu', 1),
 ('virtualasstftw', 1),
 ('fuuuuuuukkkhhhhh', 1),
 ('yuuuuuuuuuuuuuuuuuuuuuuuu', 1),
 ('sharradha', 1),
 ('mommmmmmmm', 1),
 ('ummmmaaaah', 1),
 ('esshurs', 1),
 ('titanfall', 1),
 ('pyy', 1),
 ('njnanum', 1),
 ('jigarthanda', 1),
 ('niceyekyahai', 1),
 ('thadathadacoolcool', 1),
 ('jatiya', 1),
 ('noooooook', 1),
 ('nugegod', 1),
 ('fkbgdhzvuwmbcxzlpouvda', 1),
 ('diplock', 1),
 ('ratagan', 1),
 ('ananthapur', 1),
 ('endchatseries', 1),
 ('cissie', 1),
 ('kilpauk', 1),
 ('vanakkam', 1),
 ('harshita', 1),
 ('uuuc', 1),
 ('whatpc', 1),
 ('bamhi', 1),
 ('favriout', 1),
 ('ultafulta', 1),
 ('stackshare', 1),
 ('baahubali', 1),
 ('bainchod', 1),
 ('cutex', 1),
 ('cappadoci', 1),
 ('happyhallo

In [195]:
# correct shorthand words or some misspell which can't be corrected by pyspellchecker
not_cov9 = check_coverage(vocab9, glo_dict)

100%|█████████████████████████████████████████████████████████████████████████| 4896/4896 [00:00<00:00, 1216114.67it/s]

In Embedding Index we have 97.57% coverage of distinct vocabulary
And we have 99.83% coverage of all text
The number of words which are not covered in word2vec resource is: 119


In [196]:
not_cov9[:40]

[('useropenreflink', 3),
 ('wachapp', 2),
 ('waitinnnnnnnn', 2),
 ('mumbaiindians', 1),
 ('hh:mm', 1),
 ('spitsvilla', 1),
 ('aaagdghsuuqiosjhsubdhuskqondjis', 1),
 ('whatham', 1),
 ('marryat', 1),
 ('ershad', 1),
 ('playcreepypedia', 1),
 ('feelingyusohot', 1),
 ('ialsowithyu', 1),
 ('virtualasstftw', 1),
 ('fuuuuuuukkkhhhhh', 1),
 ('yuuuuuuuuuuuuuuuuuuuuuuuu', 1),
 ('sharradha', 1),
 ('mommmmmmmm', 1),
 ('ummmmaaaah', 1),
 ('esshurs', 1),
 ('titanfall', 1),
 ('pyy', 1),
 ('njnanum', 1),
 ('jigarthanda', 1),
 ('niceyekyahai', 1),
 ('thadathadacoolcool', 1),
 ('jatiya', 1),
 ('noooooook', 1),
 ('nugegod', 1),
 ('fkbgdhzvuwmbcxzlpouvda', 1),
 ('diplock', 1),
 ('ratagan', 1),
 ('ananthapur', 1),
 ('endchatseries', 1),
 ('cissie', 1),
 ('kilpauk', 1),
 ('vanakkam', 1),
 ('harshita', 1),
 ('uuuc', 1),
 ('whatpc', 1)]

# 6. data storage

In [197]:
# finally we have 2 versions of preprocessing of data so far
# first one is preprocessed train data without spelling and shorthand words correction, 
# but after assigning space, correction of common shorthand words, adding missing charactors, removal of stopwords, emoji, emoticon and punctuations
# the second one is preprocessed train data with spelling and shorthand words correction.
# and I also have extracted all the emoji and emoticons of the data, which can be later processed for further analysis

In [198]:
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [199]:

save_path6 = "pre_{a}_all_no_correct_{b}.pkl".format(a=path_change,b=version_change)
save_path3_9 = "pre_{a}_3turn_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_patha_9 = "pre_{a}_all_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path6 ,text_all_6)
save_data(save_path3_9,text_3_9)
save_data(save_patha_9,text_all_9)